In [33]:
import spacy
import pickle
import random
import json
from spacy.util import filter_spans
from tqdm import tqdm

In [34]:
train_data = json.load(open('./data/training/train_data.json','rb')) 

In [57]:
list_an=[]

for text,annotation in train_data:
    list_an.append(list(annotation.values()))

In [36]:
nlp=spacy.load("en_core_web_lg")

In [37]:
from spacy.util import filter_spans

for txt,annot in train_data:
    doc=nlp.make_doc(txt)
    print(list(annot.values()))

In [38]:
# from spacy.util import filter_spans
# from tqdm import tqdm
# for sample_training in tqdm(train_data['annotations']):
#     txt=sample_training['text']
#     labels=sample_training['entities']
#     doc=nlp.make_doc(txt)
#     for start,end,label in labels:
#         print(start,end,label)

# doc = nlp("This is a sentence.")
# doc2=nlp("lalala la ha hu")
# spans = [doc2[0:2],doc2[1:4],doc2[1:3],doc[0:2], doc[0:2], doc[2:4],doc2[0:1]]
# filtered = filter_spans(spans)
# print(spans)
# print(filtered)

from spacy.util import filter_spans
for sample_training in annotation.values():
    print(sample_training)
    

[[587, 669, 'Skills'], [512, 553, 'College Name'], [503, 510, 'Degree'], [177, 193, 'Designation'], [34, 58, 'Companies worked at'], [14, 30, 'Designation'], [0, 13, 'Name']]


Remove overlap entyties


In [50]:
# and (list_an[j][0][i][1]<=list_an[j][0][i+1][1])
list_remov=[]
for j in range(len(list_an)):
    for i in range(len(list_an[j][0])-2):
        if((list_an[j][0][i][0] >= list_an[j][0][i+1][0]) ):
            print(list_an[j][0][i])
            print(list_an[j][0][i+1])
            list_remov.append(list_an[j][0][i])       
'''(1417, 1423, 'Companies worked at')' and '(1356, 1793, 'Skills')'''
len(list_remov)

count=0
for text, ano in train_data:
    #print(ano.values()) 
    for i in ano.values():
        for j in i:
            if(j in list_remov):
                count+=1
                i.remove(j)
count

0

In [51]:
for text, ano in train_data:
    #print(ano.values()) 
    for i in ano.values():
        for j in i:
            if(j in list_remov):
                print('Ex')

In [52]:
nlp=spacy.blank('en')

if 'ner' not in nlp.pipe_names:
    ner=nlp.create_pipe('ner')
    nlp.add_pipe('ner',last=True)

for _,annotation in train_data:
    for ent in annotation['entities']:
        ner.add_label(ent[2])
other_pipes=[pipe for pipe in nlp.pipe_names if pipe !='ner']

ner.labels

('College Name',
 'Companies worked at',
 'Degree',
 'Designation',
 'Email Address',
 'Location',
 'Name',
 'Skills',
 'Years of Experience')

In [53]:
from spacy.training import Example
with nlp.disable_pipes(*other_pipes):
    optimizer = nlp.begin_training()
    for batch in spacy.util.minibatch(train_data,size=2):
        random.shuffle(train_data)
        losses={}
        index=0
        for text, annotations in batch:
            doc=nlp.make_doc(text)
            # print('BEFORE: ',annotations['entities'])
            # annotations['entities']=filter_spans([annotations['entities']])
            # print('AFTER: ',annotations['entities'])
            
            example=Example.from_dict(doc,annotations)     
            try:
                nlp.update([example],
                drop=0.1,
                sgd=optimizer,
                losses=losses
                )
            except Exception as e:
                print(e)
                pass
        print("LOSSES: ",losses)

LOSSES:  {'ner': 136.0509955883026}
LOSSES:  {'ner': 2654.4913761019707}
LOSSES:  {'ner': 248.00915092229843}
LOSSES:  {'ner': 1422.976767539978}
LOSSES:  {'ner': 363.04590862989426}
LOSSES:  {'ner': 17.793806751898956}
LOSSES:  {'ner': 7.735190449739093}
LOSSES:  {'ner': 8.0093088534652}
LOSSES:  {'ner': 7.9576182922252485}
LOSSES:  {'ner': 8.032414857785625}
LOSSES:  {'ner': 7.986526742031359}
LOSSES:  {'ner': 7.96301423932448}
LOSSES:  {'ner': 7.961725410710642}
LOSSES:  {'ner': 8.110388010420138}
LOSSES:  {'ner': 8.116100307757733}
LOSSES:  {'ner': 7.861883743640647}
LOSSES:  {'ner': 7.865553314807357}
LOSSES:  {'ner': 7.464850562899301}
LOSSES:  {'ner': 7.879219527877922}
LOSSES:  {'ner': 7.107734176712654}
LOSSES:  {'ner': 7.471228843622157}
LOSSES:  {'ner': 5.658395021786646}
LOSSES:  {'ner': 6.081883213077276}
LOSSES:  {'ner': 5.610488586813787}
LOSSES:  {'ner': 258.13185499852625}
LOSSES:  {'ner': 211.9488467424826}
LOSSES:  {'ner': 10.411840020472482}
LOSSES:  {'ner': 224.268

ValueError: [E103] Trying to set conflicting doc.ents: '(0, 15, 'Name')' and '(0, 4, 'Location')'. A token can only be part of one entity, so make sure the entities you're setting don't overlap. To work with overlapping entities, consider using doc.spans instead.

Save model

In [54]:
nlp.to_disk('nlp_model')

Load the saved model into nlp_model

In [55]:
nlp_model=spacy.load('nlp_model')

In [56]:
print(train_data[15][0])
doc=nlp_model(train_data[15][0])
for ent in doc.ents:
    print(f'{ent.label_.upper():{30}}-{ent.text}')

Manisha Bharti Software Automation Engineer  Pune, Maharashtra - Email me on Indeed: indeed.com/r/Manisha-Bharti/3573e36088ddc073  • 3.5 years of professional IT experience in Banking and Finance domain and currently working as Software Automation Engineer in Infosys Limited, Pune. • Have experience in accounts and customers domain in banking. • Woking on SOA technology. • Hands on experience of 2+ years in Oracle 11g • 2.9 years of professional experience in Middleware Testing and Functional Testing • 4 months of experience with UiPath. • Experience on GUI and API testing on HP UFT • Working on agile methodology where involved as a senior tester. • Involved in various STLC stages including Test Planning, Test analysis, Test Execution, Defect management and Test reporting. • Possess sound knowledge of SQL, STLC, Testing Procedures, HP ALM, HP UFT, HP SV, SOAP UI, JIRA, JENKINS, CICD, UiPath. • Involved in various client presentation.  Training & Achievement  Title: Infosys E&R Training